<h1 style="color: blue">HD Wallet</h1>

## TOC:
* [Mnemonic Seed](#mneumonic-seed)
    * [Entropy](#entropy)
    * [Mnemonic creation](#mnemonic-create)
    * [Seeding](#seeding)
    * [Validation](#validation)
        * [Mnemonic validation](#mnemonic-validation)
        * [Seed validation](#seed-validation)
* [Extended Keys](#extended-keys)
    * [Master Extended Keys](#master-extended-keys)
    * [Key derivation](#key-derivation)
        * [Normal child extended private key](#normal-child-private)
        * [Hardened child extended private key](#hardened-child-private)
        * [Normal child extended public key](#normal-child-public)
        * [Hardened child extended public key](#hardened-child-public)
    * [Key derivation paths](#derivation-paths)
    * [Two parties key derivation test](#two-parties)
* [Encodings](#encodings)
    * [Extended key serialization](#extended-serialization)
        * [Private key](#prv-key-serialization)
        * [Public key](#pub-key-serialization)
    * [WIF](#wif)
    * [Addresses](#addreses)
* [Our HD-Wallet](#our-wallet)
    

In [795]:
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat, PrivateFormat
from cryptography.hazmat import primitives
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes, hmac
from bitarray import bitarray, util
from secrets import token_bytes

from ecpy.curves import Curve,Point

# Mneumonic seed <a class="anchor" id="mneumonic-seed"></a>

## Entropy <a clas="anchor" id="entropy"></a>
Generate a blob between 128 and 256 bits from a secure random source.
The length of the entropy should be multiple of 32.

In [796]:
def entropy_gen(n = 128):
    if n < 128 or n > 256:
        raise Exception("Entropy size must be between 128 and 256 bits inclusive.")
    if n % 32 != 0:
        raise Exception("Entropy size must be multiple of 32.")
    return token_bytes(n // 8)

entropy = entropy_gen(32*8)
print(f"Entropy (hex): {entropy.hex()}") #256 bits

Entropy (hex): 0d86a3f30156f0f192b0d8b6de40560eb11cef7550c058a76c737dd6136f01c5


## Mnemonic creation <a class="anchor" id="mnemonic-create"></a>

To make the entropy more human friendly we conver it to a series or words and introduce some error checking. To do so in this case we are gonna make use of __[BIP39-english](https://raw.githubusercontent.com/otromimi/bitcoin_testpad/wallet/BIP-0039_english.txt)__.

<div style="color:orange">
<h3>Atention</h3>
Mnemonic phrases are suported in many lenguages and alphabets. This notebook only uses the english ASCII variant.</br>
For other alphabets use UTF-8 NFKD encoding.
</div>

In [797]:
colors = {'purple':'\033[95m', 'red':'\033[91m', 'yellow':'\033[93m', 'green':'\033[92m', 'blue':'\033[94m'}

def mnemonic(bits):
    # Entorpy 2 binary
    entropy_bits = bitarray()
    entropy_bits.frombytes(bits)
    
    # Checksum
    hash256 = hashes.Hash(hashes.SHA256())
    hash256.update(bits)
    bin_h = bitarray()
    bin_h.frombytes(hash256.finalize())
    
    entropy_bits = entropy_bits + bin_h[:(len(entropy_bits) // 32)]

    numbers = [] # words numbers
    for i in range(len(entropy_bits)//11):
        numbers.append(util.ba2int(entropy_bits[11*i:11*i+11]))

    words = {} # words in our entropy
    with open('BIP-0039_english.txt', 'r') as file:
        for i, word in enumerate(file):
            if i in numbers:
                words[i] = word
                if len(numbers) == len(words):
                    break
    
    mnemonic_sentence = [] # mneumonic
    for i in numbers:
        mnemonic_sentence.append(words[i].strip()) 
    
    return tuple(mnemonic_sentence)


# change this assigment to check for different values.
entropy = entropy # Ex: entropy = entropy_gen()

mnemonic_sentence = " ".join(mnemonic(entropy))

# Entropy binary
entropy_bits = bitarray()
entropy_bits.frombytes(entropy)

# Entropy hash
hash256 = hashes.Hash(hashes.SHA256())
hash256.update(entropy)
bin_h = bitarray()
entropy_hash = hash256.finalize()
bin_h.frombytes(entropy_hash)

print(colors['purple']+entropy_bits.to01(), end="")
print(colors['yellow']+bin_h.to01()[:(len(entropy_bits) // 32)], end="\n")
print(f"{colors['purple']}\u2589 {'Entropy'}")
print(f"{colors['yellow']}\u2589 {'Checksum (hash)'}\033[0m\n")

print(f"Entropy (hex): {entropy.hex()}")
print(f"entropy SHA-256: {entropy_hash.hex()}")

print("\n\u250F"+"\u2501"*(len(mnemonic_sentence)+2)+"\u2513")
print("\u2503 "+mnemonic_sentence+" \u2503")
print("\u2517"+"\u2501"*(len(mnemonic_sentence)+2)+"\u251B")

del entropy_bits, hash256, bin_h, entropy_hash


000011011000011010100011111100110000000101010110111100001111000110010010101100001101100010110110110111100100000001010110000011101011000100011100111011110111010101010000110000000101100010100111011011000111001101111101110101100001001101101111000000011100010100100111
▉ Entropy
▉ Checksum (hash)

Entropy (hex): 0d86a3f30156f0f192b0d8b6de40560eb11cef7550c058a76c737dd6136f01c5
entropy SHA-256: 27b2173527c549535ea3cb6ef5a497e2fcb0276bcd5f809c801af9823623008f

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ assault crush wolf actor humor juice enlist brand replace velvet better buddy balcony jeans stem blossom flavor derive broom wing gentle resource alpha pilot ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


## Seeding<a class="anchor" id="seeding"></a>
Derivates the seed that will be use for creating our cryptographic keys. We achive this by using an algorithm computer intensive that will slow down an attack.

Algorithm used: PBKDF2_HMAC (Password Based Key Derivation Function 2)</br>
* Hashing function: SHA-512 HMAC</br>
* Iterations: 2048</br>
* length: resulting key (seed) lenght in bytes</br>

Thanks to the use of a HMAC (Hash Message Authentication Code) hash function we can add a salt (password) to our nmonic phrase.



In [798]:
def mnemonic2seed(mnemonic_sentence, salt=""):
    
    mnemonic_dev = PBKDF2HMAC(
        algorithm = hashes.SHA512(),
        length = 64, #bytes
        salt = ("mnemonic"+salt).encode('ascii'),
        iterations = 2048
    )

    mnemonic_bytes = mnemonic_sentence.encode('ascii')
    seed = mnemonic_dev.derive(mnemonic_bytes)

    return seed


recovery_pass = "" # In case we want to add a salt to our mneumonic, add in here

seed = mnemonic2seed(mnemonic_sentence, recovery_pass)

if recovery_pass:
    print(f"Recovery password: {colors['red']}{recovery_pass}\033[0m")
print(f"Seed: {seed.hex()}")

Seed: f12b75f95750b76b98ff4aacf1aa576c1b150de2ab3bcf24711a1ee78caf256127dd62b791838c3a4a766de216ad5f9ee54102f5aa343cddadf7defd132596cf


## Validation<a class="anchor" id="validation"></a>

### Mnemonic validation<a class="anchor" id="mnemonic-validation"></a>
Validates de entropy against its hash to determinate if the mnemonic secuence was correct.

Requires: __[BIP39-english](https://raw.githubusercontent.com/otromimi/bitcoin_testpad/wallet/BIP-0039_english.txt)__.

In [799]:
def mnemonic_check(mnemonic):

    words = mnemonic.split(" ")
    numbers = [None] * len(words)

    with open('BIP-0039_english.txt') as file:
        for i, word in enumerate(file):
            if word.strip() in words:
                for j, item in enumerate(words):
                    if item == word.strip():
                        numbers[j] = i
    
    entropy = "".join([f'{i:011b}' for i in numbers])
    entropy = bitarray(entropy)

    checksum_len = len(entropy) % 32

    hash256 = hashes.Hash(hashes.SHA256())
    hash256.update(entropy[:-checksum_len].tobytes())
    bin_h = bitarray()
    entropy_hash = hash256.finalize()
    bin_h.frombytes(entropy_hash)
    
    if entropy[-checksum_len:] != bin_h[:checksum_len]:
        # Just raised in case checksum for mneumonic fails.
        raise Exception(f"Checksum fail, hash: {entropy_hash.hex()}")
    
    return entropy[:-checksum_len].tobytes()



recover_entropy = mnemonic_check(mnemonic_sentence)

print(f"{entropy.hex()} -> entropy") # entropy generated on first cell
print(f"{recover_entropy.hex()} -> recovered entropy")


0d86a3f30156f0f192b0d8b6de40560eb11cef7550c058a76c737dd6136f01c5 -> entropy
0d86a3f30156f0f192b0d8b6de40560eb11cef7550c058a76c737dd6136f01c5 -> recovered entropy


### Seed validation<a class="anchor" id="seed-validation"></a>

Validates the seed against the entropy and password.

In [800]:
def seed_check(seed, password):

    seed_chk = PBKDF2HMAC(
        algorithm = hashes.SHA512(),
        length = 64,
        salt = ("mnemonic"+password).encode('ascii'),
        iterations = 2048
    )
    seed_chk.verify(mnemonic_sentence.encode('ascii'), seed) # in case validation fails will raise a exception

if not seed_check(seed, recovery_pass):
    print("Seed check passed. \u2705")

Seed check passed. ✅


***

# Extended keys <a class="anchor" id="extended-keys"></a>

## Master Extended Keys <a class="anchor" id="master-extended-keys"></a>


In [801]:
def master_key_extended(seed):

    expression = "Bitcoin seed" # Used for master key

    hmac_master = hmac.HMAC(expression.encode('ascii'), hashes.SHA512())
    hmac_master.update(seed)
    output = hmac_master.finalize()

    secret_num = output[:32]
    chain_code = output[32:]

    return ('m', secret_num, chain_code)


#prv_master_key, chain_master_code = master_key_extended(seed)



test_seed = bytes.fromhex("67f93560761e20617de26e0cb84f7234aaf373ed2e66295c3d7397e6d7ebe882ea396d5d293808b0defd7edd2babd4c091ad942e6a9351e6d075a29d4df872af")

index, prv_master_key, chain_master_code = master_key_extended(test_seed)

private_key = ec.derive_private_key(int.from_bytes(prv_master_key, 'big'), ec.SECP256K1())
public_key = private_key.public_key()
pub_master_key = public_key.public_bytes(Encoding.X962, PublicFormat.CompressedPoint)


print(f"Chain code: {chain_master_code.hex()}")
print(f"Private master key (d): {prv_master_key.hex()}")
print(f"Public master key (compressed): {pub_master_key.hex()}")

del index

Chain code: 463223aac10fb13f291a1bc76bc26003d98da661cb76df61e750c139826dea8b
Private master key (d): f79bb0d317b310b261a55a8ab393b4c8a1aba6fa4d08aef379caba502d5d67f9
Public master key (compressed): 0252c616d91a2488c1fd1f0f172e98f7d1f6e51f8f389b2f8d632a8b490d5f6da9



## Key derivation<a class="anchor" id="key-derivation"></a>

1. [Normal child extended private key](#normal-child-private)
2. [Hardened child extended private key](#hardened-child-private)
3. [Normal child extended public key](#normal-child-public)
4. [Hardened child extended public key](#hardened-child-public)

In [802]:
def key_derivation(index, parent_key, parent_chain_code):

    index_bytes = index.to_bytes(32 // 8, 'big')
    data = parent_key + index_bytes
    salt = parent_chain_code

    hmac_512 = hmac.HMAC(salt, hashes.SHA512())
    hmac_512.update(data)
    output = hmac_512.finalize()

    return (output[:32], output[32:])

### Normal child extended private key<a class="anchor" id="normal-child-private"></a>

In [803]:
def private_normal_child(index, parent_prv_key, parent_chain_code):
    cv = Curve.get_curve('secp256k1')

    # Order of SECP256K1
    n = cv.order

    # Index range checking
    if index > 2147483647 | index < 0:
        raise f"Index: {index}, out of range for soft derivation."
    
    # Generating public key
    private_key = ec.derive_private_key(int.from_bytes(parent_prv_key, 'big'), ec.SECP256K1())
    public_key = private_key.public_key()
    parent_pub_key = public_key.public_bytes(Encoding.X962, PublicFormat.CompressedPoint)

    # Derivation function
    child_prv_key, child_chain_code = key_derivation(index, parent_pub_key, parent_chain_code)

    # Checking child_chain_code
    if int.from_bytes(child_chain_code, 'big') > n:
        raise f"Chain code bigger than curve order \"n\"; Try with index {index + 1}"
    
    # Calculating child private key
    child_prv_key = ((int.from_bytes(child_prv_key, 'big') + int.from_bytes(parent_prv_key, 'big')) % n).to_bytes(32, 'big')

    return (index, child_prv_key, child_chain_code)




index, key, chain_code = private_normal_child(0, prv_master_key, chain_master_code)

private_key = ec.derive_private_key(int.from_bytes(key, 'big'), ec.SECP256K1())
public_key = private_key.public_key()
public_compress_child = public_key.public_bytes(Encoding.X962, PublicFormat.CompressedPoint)

print(f"Index: {index}")
print(f"Chain code: {chain_code.hex()}")
print(f"Private key (d): {key.hex()}")
print(f"Public key (compressed): {public_compress_child.hex()}")

del index, key, chain_code, private_key, public_key, public_compress_child

Index: 0
Chain code: 05aae71d7c080474efaab01fa79e96f4c6cfe243237780b0df4bc36106228e31
Private key (d): 39f329fedba2a68e2a804fcd9aeea4104ace9080212a52ce8b52c1fb89850c72
Public key (compressed): 030204d3503024160e8303c0042930ea92a9d671de9aa139c1867353f6b6664e59


### Hardened child extended private key<a class="anchor" id="hardened-child-private"></a>

In [804]:
def private_hardened_child(index, parent_prv_key, parent_chain_code):
    cv = Curve.get_curve('secp256k1')

    # Order of SECP256K1
    n = cv.order

    # Index range checking
    if index > 2147483647 | index < 0:
        raise f"Index: {index}, Hard index: {index + 2147483648}, out of range for hard derivation."
    
    # Generating public key --> No required

    # Derivation function
    child_prv_key, child_chain_code = key_derivation(
        index + 2147483648, # Hard index range
        b'\x00' + parent_prv_key, # Leading bit for public key compensation
        parent_chain_code
    )

    # Checking child_chain_code
    if int.from_bytes(child_chain_code, 'big') > n:
        raise f"Chain code bigger than curve order \"n\"; Try with index {index + 1}"
    
    # Calculating child private key
    child_prv_key = ((int.from_bytes(child_prv_key, 'big') + int.from_bytes(parent_prv_key, 'big')) % n).to_bytes(32, 'big')

    return (index, child_prv_key, child_chain_code)




index, key, chain_code = private_hardened_child(0, prv_master_key, chain_master_code)

private_key = ec.derive_private_key(int.from_bytes(key, 'big'), ec.SECP256K1())
public_key = private_key.public_key()
public_compress_child = public_key.public_bytes(Encoding.X962, PublicFormat.CompressedPoint)


print(f"Index: {index}, Hard index: {index + 2147483648}")
print(f"Chain code: {chain_code.hex()}")
print(f"Private key (d): {key.hex()}")
print(f"Public key (compressed): {public_compress_child.hex()}")

del index, key, chain_code, private_key, public_key, public_compress_child

Index: 0, Hard index: 2147483648
Chain code: cb3c17166cc30eb7fdd11993fb7307531372e565cd7c7136cbfa4655622bc2be
Private key (d): 7272904512add56fef94c7b4cfc62bedd0632afbad680f2eb404e95f2d84cbfa
Public key (compressed): 0355cff4a963ce259b08be9a864564caca210eb4eb35fcb75712e4bba7550efd95


### Normal child extended public key<a class="anchor" id="normal-child-public"></a>

In [805]:
def public_normal_child(index, parent_pub_key, parent_chain_code):
    cv = Curve.get_curve('secp256k1')

    # Order of SECP256K1
    n = cv.order

    # Index range checking
    if index > 2147483647 | index < 0:
        raise f"Index: {index}, out of range for soft derivation."
    
    # Generating public key --> No required, we pass the pub key as parameter

    # Derivation function
    child_pub_key, child_chain_code = key_derivation(index, parent_pub_key, parent_chain_code)

    # Checking child_chain_code
    if int.from_bytes(child_chain_code, 'big') > n:
        raise f"Chain code bigger than curve order \"n\"; Try with index {index + 1}"

    # Calculating child private key
    parent_point = cv.decode_point(parent_pub_key)
    child_point = cv.mul_point(int.from_bytes(child_pub_key, 'big'), cv.generator)
    result_point = parent_point + child_point

    # Encoding in weird format because ECPy doesn't support compress point encoding
    child_pub_key = ec.EllipticCurvePublicKey.from_encoded_point(
        ec.SECP256K1(), 
        bytes(cv.encode_point(result_point))).public_bytes(
            Encoding.X962, 
            PublicFormat.CompressedPoint
        )
    
    return (index, child_pub_key, child_chain_code)




index, key, chain_code = public_normal_child(0, pub_master_key, chain_master_code)

print(f"Index: {index}")
print(f"Chain code: {chain_code.hex()}")
print(f"Parent public key (compressed): {pub_master_key.hex()}")
print(f"Public key (compressed): {key.hex()}")

del index, key, chain_code

Index: 0
Chain code: 05aae71d7c080474efaab01fa79e96f4c6cfe243237780b0df4bc36106228e31
Parent public key (compressed): 0252c616d91a2488c1fd1f0f172e98f7d1f6e51f8f389b2f8d632a8b490d5f6da9
Public key (compressed): 030204d3503024160e8303c0042930ea92a9d671de9aa139c1867353f6b6664e59


### Hardened child extended public key<a class="anchor" id="hardened-child-public"></a>

In [806]:
def public_hardened_child(index, parent_pub_key, parent_chain_code):
    raise f"You need the private key to derive its peer public key."


## Key derivation paths <a class="anchor" id="derivation-paths"></a>

In [828]:
def derive_path(path, seed):
    path_list = path.split('/')
    index, key, chain_code = None, None, None
    for term in path_list:
        if term == 'm':
            # Master extended key from seed
            _, key, chain_code = master_key_extended(seed)
            continue
        if "'" in term:
            # Case term is hard
            _, key, chain_code = private_hardened_child(int(term.strip("'")), key, chain_code)
        else:
            # Case the term is soft of normal
            _, key, chain_code = private_normal_child(int(term), key, chain_code)
    return key, chain_code

derivation_path = "m/44'/0'/0'/0/0" # Path extipulated on BIP 44

path_key, _ = derive_path(derivation_path, seed)

print(f"Private key on {derivation_path}:\n\t{path_key.hex()}")

Private key on m/44'/0'/0'/0/0:
	f850b6df12f144c270cb63e0d16ee149da6a4d13cc7b5295d9d5a399a78f5f2e


## Two parties key derivation test <a class="anchor" id="two-parties"></a>

For this test we are gonna derive in parallel the public and private key; and then sign a message with those keys independently derived.

In [827]:
# Finding the master pub, prv and chain-code
prv_secret, chain_code_m = derive_path("m", seed)
prv_key_m = ec.derive_private_key(int.from_bytes(prv_secret, 'big'), ec.SECP256K1())
pub_key_m = prv_key_m.public_key().public_bytes(Encoding.X962, PublicFormat.CompressedPoint)

# Derive our path for our private key
prv_secret, _ = derive_path("m/444/8", seed) # paths must be the same
prv_key = ec.derive_private_key(int.from_bytes(prv_secret, 'big'), ec.SECP256K1())

# We derive the same path for our public key
_, pub_key, chain_code_child = public_normal_child(444, pub_key_m, chain_code_m) # 444/
_, pub_key, chain_code_child = public_normal_child(8, pub_key, chain_code_child) # 8/
pub_key = ec.EllipticCurvePublicKey.from_encoded_point(ec.SECP256K1(), pub_key)

# We sign a message with our derived private key
message = b"Testing derivaiton of extended keys."
signature = prv_key.sign(message, ec.ECDSA(hashes.SHA256()))

# If no exception raised then the signature is correct
pub_key.verify(signature, message, ec.ECDSA(hashes.SHA256()))


del prv_secret, prv_key_m, pub_key_m, prv_key, pub_key, message, signature

***

# Encodings <a class="anchor" id="encodings"></a>